# Combining datasets

In this notebook, we are going to use the [Kaggle competition](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) dataset on estimating property sales prices in Melbourne, Australia.

We will use the reduced dataset produced by [DanB](https://www.kaggle.com/dansbecker).

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [3]:
import plotly
plotly.__version__

'5.5.0'

In [4]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [5]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Additional data

We are going to augment the data present in the given set with a similar dataset: the AirBnB platform posts in Melbourne in the year 2018. The goal is to more accurately estimate the neighborhood value of each property.

To do this, we'll use [a dataset](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) of *scrapings* of the site by [Tyler Xie]( https://www.kaggle.com/tylerx), also available in a Kaggle competition.

In [6]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


In [8]:
len(airbnb_df)

22895

In order to perform this combination of data, we have to find a column that contains information in common. We will use the zipcode.

* What are the disadvantages of using the zipcode to add information about the neighborhood of each property for sale? Does it correspond to what we intuitively understand as neighborhood?
* What other methods could be used?

In [9]:
airbnb_df.zipcode.value_counts()[:5]

3000.0    2491
3000       876
3006.0     860
3182.0     579
3182       556
Name: zipcode, dtype: int64

However, this zipcode has different types, which we will need to standardize before proceeding with merging the data.

In [10]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

### Common values

Before joining the data sets, we need to make sure we have enough records in common to aggregate relevant information.

In [11]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

Airbnb unique zipcodes 248
Sales unique zipcodes 198
Common zipcodes 191


In [12]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.9985272459499264
Records in Airbnb df with corresponding zipcode form Sales df 0.9302904564315353


### Visual Exploration

To finish making sure that the areas represented by both datasets are consistent, we can plot the coordinates on a map using Plotly.

Although these visualizations are not complete in terms of the geographical information they show, we can roughly see that they are similar areas.

In [13]:
import plotly.express as px
data = melb_df.sample(300)
fig = px.scatter_geo(
    data, lat=data.Lattitude, lon=data.Longtitude, color=data.YearBuilt
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [14]:
import plotly.express as px
color_col = 'city'
data = airbnb_df[~airbnb_df[color_col].isna()].sample(300)
fig = px.scatter_geo(
    data, lat=data.latitude, lon=data.longitude, color=data[color_col]
    )
fig.update_geos(fitbounds="locations")
fig.show()

## Merge Operation

It's time to combine both data sets. A frequently tempting option is to apply the merge operation without analyzing the situation too much. However, this operation can cause an **exponential growth** of our data set!

Why does the following operation generate a DataFrame with 2 million rows?

In [15]:
## Incorrect!
len(melb_df.merge(airbnb_df, how='left',
                  left_on='Postcode', right_on='zipcode'))

2139684

Let's remember that our goal is to add information about the property's neighborhood value. What information from the set of airbnb helps us to represent this?

* Average `price` per zipcode
* Average `weekly_price` per zipcode
* Average `monthly_price` per zipcode

It is up to you to decide which of all of them to use.

In [16]:
relevant_cols = ['price', 'weekly_price', 'monthly_price']

When we want to add a single data after grouping a DataFrame, the operation is quite simple. You just have to keep in mind that the name of the columns will be the same, even though their value is now added.

In [17]:
# Option 1: inefficient form.
# The aggregation is calculated over all columns, and after is calculated
# we filter only the columns we want.
# airbnb_df.groupby('zipcode').mean()[relevant_cols].reset_index()[:5]

# Option 2: efficient form
# First we filter out the columns we want, and then we only aggregate
# those. Be careful to include the grouping column as well.
airbnb_df[relevant_cols + ['zipcode']]\
  .groupby('zipcode').mean().reset_index()[:5]

,zipcode,price,weekly_price,monthly_price
0,2010.0,40.000000,NaN,NaN
1,2134.0,50.000000,NaN,NaN
2,2582.0,104.000000,NaN,NaN
3,3000.0,150.504307,918.738956,3407.204651
4,3001.0,132.500000,NaN,NaN


Grouping and adding using more than one type of operation is a bit more complicated. Pandas provides us with many ways to write the same operation. None of them are very intuitive, and usually require renaming the columns to give them more meaningful names.

In [18]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

## Option 2
# Pass as argument name the new name of the column, and as value a tuple where
# the first value is the original column and the second value is the operation.
# airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
#   .agg(airbnb_record_count=('price', 'count'),
#        airbnb_price_mean=('price', 'mean'),
#        airbnb_weekly_price_mean=('weekly_price', 'mean'),
#        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
#   .reset_index()

airbnb_price_by_zipcode[:3]

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.0,1,NaN,NaN
1,2134.0,50.0,1,NaN,NaN
2,2582.0,104.0,1,NaN,NaN


In [19]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
4802,Port Melbourne,176 Dow St,3,h,1080000.0,S,Biggin,30/07/2016,3.8,3207.0,...,Port Phillip,-37.84010,144.94300,Southern Metropolitan,8648.0,3207.0,200.210117,257.0,1480.790698,4743.964286
10526,Sunshine North,14 Oxford St,3,h,905000.0,SP,Douglas,27/05/2017,10.5,3020.0,...,Brimbank,-37.77004,144.84254,Western Metropolitan,4217.0,3020.0,69.555556,45.0,160.000000,600.000000
4371,North Melbourne,509/30 Wreckyn St,2,u,620000.0,S,LITTLE,19/11/2016,2.3,3051.0,...,Melbourne,-37.80040,144.95440,Northern Metropolitan,6821.0,3051.0,112.233728,338.0,753.527778,3152.153846
7883,Glen Iris,2/15 Scott Gr,3,t,1253000.0,S,Abercromby's,8/04/2017,9.2,3146.0,...,Stonnington,-37.85740,145.04940,Southern Metropolitan,10412.0,3146.0,135.833333,72.0,770.111111,2940.750000
12884,Yarraville,23 Gray St,2,h,921000.0,S,hockingstuart,16/09/2017,6.3,3013.0,...,NaN,-37.81899,144.89507,Western Metropolitan,6543.0,3013.0,135.609756,82.0,923.571429,2966.090909


In [20]:
merged_sales_df.to_csv("melb_data_extended.csv", index=None)

In [21]:
from google.colab import files
files.download('melb_data_extended.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>